## machine learning model trial for predicting rigid particles

In [1]:
import os
import glob
import matplotlib               # type: ignore
import numpy             as np  # type: ignore
import matplotlib.pyplot as plt # type: ignore
import matplotlib.colors as mcolors
import platform
from   pathlib           import Path
import importlib
import readFiles
from tqdm import tqdm
import networkx as nx
importlib.reload(readFiles)

# ML
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

# Matplotlib rc parameters modification
plt.rcParams.update({
  "figure.max_open_warning" : 0,
  "text.usetex"             : True,
  "text.latex.preamble"     : r"\usepackage{amsmath, bm, type1cm}",  # Added \bm for bold math
  "figure.autolayout"       : True,
  "font.family"             : "STIXGeneral",
  "mathtext.fontset"        : "stix",
  "font.size"               : 8,
  "xtick.labelsize"         : 8,
  "ytick.labelsize"         : 8,
  "lines.linewidth"         : 1,
  "lines.markersize"        : 5,
})
#plt.rcParams['text.latex.preamble']= r"\usepackage{amsmath}"
matplotlib.rc('text', usetex=True)
matplotlib.rcParams['text.latex.preamble'] = r'\boldmath'

colors = ['#4a91b5', '#e68139', '#5da258', '#87629b', '#1b9e77']


if platform.system() == 'Darwin':  # macOS
    topDir = Path("/Volumes/rahul_2TB/high_bidispersity/new_data/")
    fig_save_path = Path("/Users/rahul/City College Dropbox/Rahul Pandare/CUNY/research/bidisperse_project/figures/ang_vel/")
elif platform.system() == 'Linux':
    topDir = Path("/media/rahul/rahul_2TB/high_bidispersity/new_data/")
    fig_save_path = Path("/media/Linux_1TB/City College Dropbox/Rahul Pandare/CUNY/research/bidisperse_project/figures/ang_vel/")
else:
    raise OSError("Unsupported OS")

In [3]:
## Input set

npp  = 1000
runs = 1
phi  = 0.74
vr   = '0.5'
ar   = 1.4 #[1.0, 1.4, 2.0, 4.0]
off  = 100
k    = 7  # max number of neighbors

phir     = f"{phi:.3f}" if phi != round(phi, 2) else f"{phi:.2f}"
dataname = f"{topDir}/NP_{npp}/phi_{phir}/ar_{ar}/Vr_{vr}"

frameArr = []
angvel_all      = []
neighAngVel_all = []
rigid_all       = []
radius_all      = []
posx_all = []
posy_all = []
idxlen = []

if os.path.exists(dataname):
    for l in range(runs):
        dataname  = f"{topDir}/NP_{npp}/phi_{phir}/ar_{ar}/Vr_{vr}/run_{l+1}"  
        dataFile  = open(glob.glob(f'{dataname}/data_*.dat')[0], 'r')
        parFile   = open(glob.glob(f'{dataname}/par_*.dat' )[0], 'r')
        rigFile   = open(glob.glob(f'{dataname}/rig_*.dat' )[0], 'r')
        datdata   = np.genfromtxt(dataFile)
        pardata   = readFiles.readParFile(parFile)
        rigdata   = readFiles.rigList(rigFile)
        srate     = datdata[off:, 2]
        totStrain = datdata[-1, 1]
        parLines  = open(glob.glob(f'{dataname}/par_*.dat' )[0], 'r').readlines()

        lx = float(parLines[3].split()[2]) 
        lz = float(parLines[5].split()[2])
            
        for i, frame in enumerate(pardata[off:]):
            angvel = frame[:,8]
            pidx   = frame[:, 0]
            pr     = frame[:, 1]
            px     = frame[:, 2]
            pz     = frame[:, 3]
            sr     = srate[i]            
            
            xmat, zmat = np.outer(px, np.ones(len(px))), np.outer(pz, np.ones(len(pz))) # broadcasting position array
            dxij, dzij = xmat.transpose() - xmat, zmat.transpose() - zmat        # distance matrix
            
            # Lees Edwards boundary:
            dxij[dzij > lz/2.]  -= sr*lx
            dzij[dzij > lz/2.]  -= lz
            
            dxij[dzij < -lz/2.] += sr*lx
            dzij[dzij < -lz/2.] += lz
            
            # X peridodic:
            dxij[dxij >  lx/2.] -= lx
            dxij[dxij < -lx/2.] += lx
        
            dij = np.sqrt(dxij**2 + dzij**2) # norm dist matrix
            
            for ii in pidx:
                sorted_indices = np.argsort(dij[:, int(ii)]) # all neighbors sorted by distance to particle ii
                within_cutoff  = dij[sorted_indices, int(ii)] <= 1.5 * (1 + float(ar)) # distance cuttoff
                idx = sorted_indices[within_cutoff][1:k+1]  # skip self # this list wont crash
                idxlen.append(len(idx))

                neighAngVel = np.mean(angvel[idx])     # mean ang vel of nearest neighbors
                neighAngVel_all.append(neighAngVel/sr) 

            rigList = [set(sum(rigFrame, [])) for rigFrame in rigdata]
            rigid = np.array([0] * npp)
            rigid[list(rigList[off+i])] = 1

            angvel_all.extend(angvel/sr)
            rigid_all.extend(list(rigid))
            radius_all.extend(pr)
            posx_all.extend(px)
            posy_all.extend(pz)
                
angvel_all      = np.array(angvel_all) - np.mean(angvel_all)
neighAngVel_all = np.array(neighAngVel_all) - np.mean(neighAngVel_all)
rigid_all       = np.array(rigid_all)
radius_all      = np.array(radius_all)
posx_all        = np.array(posx_all)
posy_all        = np.array(posy_all)

In [12]:
rigid = np.array([0] * npp)

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

# Stack features
X = np.stack([angvel_all, neighAngVel_all, radius_all, posx_all, posy_all], axis=1)
y = rigid_all

# Use 1 million points
subset_size = 1_000_000
indices = np.random.choice(len(X), size=subset_size, replace=False)
X = X[indices]
y = y[indices]

# Normalize
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Stratified split
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Convert to tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_val   = torch.tensor(X_val, dtype=torch.float32)
y_val   = torch.tensor(y_val, dtype=torch.float32).view(-1, 1)

# Compute class weights for imbalance
pos_weight_value = ((len(y_train) - y_train.sum()) / y_train.sum()).item()
pos_weight = torch.tensor(pos_weight_value, dtype=torch.float32)

# Define neural network (without sigmoid!)
class RigidClusterNet(nn.Module):
    def __init__(self):
        super(RigidClusterNet, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(5, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)  # no Sigmoid here!
        )

    def forward(self, x):
        return self.net(x)

model = RigidClusterNet()
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training
epochs = 10
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    output = model(X_train)
    loss = criterion(output, y_train)
    loss.backward()
    optimizer.step()

    # Validation
    model.eval()
    with torch.no_grad():
        val_logits = model(X_val)
        val_output = torch.sigmoid(val_logits)  # apply sigmoid here!
        val_loss = criterion(val_logits, y_val)
        val_pred = (val_output > 0.5).float()
        val_acc = (val_pred == y_val).float().mean()
        tp = ((val_pred == 1) & (y_val == 1)).sum().item()
        fp = ((val_pred == 1) & (y_val == 0)).sum().item()
        fn = ((val_pred == 0) & (y_val == 1)).sum().item()
        precision = tp / (tp + fp + 1e-6)
        recall = tp / (tp + fn + 1e-6)

    print(f"Epoch {epoch+1}: Train Loss={loss.item():.4f}, Val Loss={val_loss.item():.4f}, "
          f"Val Acc={val_acc.item():.4f}, Precision={precision:.4f}, Recall={recall:.4f}")

Epoch 1: Train Loss=1.2391, Val Loss=1.2360, Val Acc=0.1158, Precision=0.1141, Recall=0.9961
Epoch 2: Train Loss=1.2358, Val Loss=1.2330, Val Acc=0.1271, Precision=0.1142, Recall=0.9822
Epoch 3: Train Loss=1.2328, Val Loss=1.2302, Val Acc=0.1748, Precision=0.1145, Recall=0.9229
Epoch 4: Train Loss=1.2301, Val Loss=1.2277, Val Acc=0.2792, Precision=0.1167, Recall=0.8075
Epoch 5: Train Loss=1.2276, Val Loss=1.2254, Val Acc=0.3993, Precision=0.1233, Recall=0.6962
Epoch 6: Train Loss=1.2253, Val Loss=1.2233, Val Acc=0.4853, Precision=0.1283, Recall=0.6041
Epoch 7: Train Loss=1.2231, Val Loss=1.2214, Val Acc=0.5465, Precision=0.1332, Recall=0.5385
Epoch 8: Train Loss=1.2212, Val Loss=1.2196, Val Acc=0.5915, Precision=0.1379, Recall=0.4900
Epoch 9: Train Loss=1.2194, Val Loss=1.2179, Val Acc=0.6248, Precision=0.1427, Recall=0.4556
Epoch 10: Train Loss=1.2177, Val Loss=1.2163, Val Acc=0.6481, Precision=0.1464, Recall=0.4300


In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
import numpy as np

# Stack features
X = np.stack([angvel_all, neighAngVel_all, radius_all, posx_all, posy_all], axis=1)
y = rigid_all

# Subset (1 million)
subset_size = 1_000_000
indices = np.random.choice(len(X), size=subset_size, replace=False)
X = X[indices]
y = y[indices]

# Normalize
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Stratified train/val split
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Convert to tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_val   = torch.tensor(X_val, dtype=torch.float32)
y_val   = torch.tensor(y_val, dtype=torch.float32).view(-1, 1)

# Compute pos_weight for BCEWithLogitsLoss
pos_weight_value = ((len(y_train) - y_train.sum()) / y_train.sum()).item()
pos_weight = torch.tensor(pos_weight_value, dtype=torch.float32)

# Define improved neural net
class RigidClusterNet(nn.Module):
    def __init__(self):
        super(RigidClusterNet, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(5, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)  # No sigmoid here
        )

    def forward(self, x):
        return self.net(x)

# Instantiate model
model = RigidClusterNet()
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# DataLoader for batch training
from torch.utils.data import DataLoader, TensorDataset

batch_size = 2048
train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=batch_size, shuffle=True)

# Training loop
epochs = 25
for epoch in range(epochs):
    model.train()
    total_loss = 0.0

    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        logits = model(batch_X)
        loss = criterion(logits, batch_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * batch_X.size(0)

    train_loss = total_loss / len(train_loader.dataset)

    # Validation
    model.eval()
    with torch.no_grad():
        val_logits = model(X_val)
        val_output = torch.sigmoid(val_logits)  # Apply sigmoid here
        val_loss = criterion(val_logits, y_val)
        val_pred = (val_output > 0.5).float()
        val_acc = (val_pred == y_val).float().mean().item()

        tp = ((val_pred == 1) & (y_val == 1)).sum().item()
        fp = ((val_pred == 1) & (y_val == 0)).sum().item()
        fn = ((val_pred == 0) & (y_val == 1)).sum().item()

        precision = tp / (tp + fp + 1e-6)
        recall = tp / (tp + fn + 1e-6)
        f1 = 2 * precision * recall / (precision + recall + 1e-6)
        auc = roc_auc_score(y_val.numpy(), val_output.numpy())

    print(f"Epoch {epoch+1:02d}: Train Loss={train_loss:.4f}, Val Loss={val_loss.item():.4f}, "
          f"Acc={val_acc:.4f}, Precision={precision:.4f}, Recall={recall:.4f}, "
          f"F1={f1:.4f}, AUC={auc:.4f}")

Epoch 01: Train Loss=1.0793, Val Loss=1.0430, Acc=0.5678, Precision=0.1855, Recall=0.8214, F1=0.3026, AUC=0.7303
Epoch 02: Train Loss=1.0556, Val Loss=1.0418, Acc=0.5646, Precision=0.1850, Recall=0.8258, F1=0.3023, AUC=0.7309
Epoch 03: Train Loss=1.0531, Val Loss=1.0404, Acc=0.5773, Precision=0.1875, Recall=0.8108, F1=0.3046, AUC=0.7327
Epoch 04: Train Loss=1.0514, Val Loss=1.0396, Acc=0.5702, Precision=0.1863, Recall=0.8205, F1=0.3036, AUC=0.7325
Epoch 05: Train Loss=1.0497, Val Loss=1.0389, Acc=0.5658, Precision=0.1855, Recall=0.8266, F1=0.3030, AUC=0.7328
Epoch 06: Train Loss=1.0490, Val Loss=1.0392, Acc=0.5643, Precision=0.1854, Recall=0.8292, F1=0.3030, AUC=0.7326
Epoch 07: Train Loss=1.0476, Val Loss=1.0390, Acc=0.5649, Precision=0.1853, Recall=0.8272, F1=0.3028, AUC=0.7329
Epoch 08: Train Loss=1.0474, Val Loss=1.0388, Acc=0.5635, Precision=0.1851, Recall=0.8298, F1=0.3027, AUC=0.7332
Epoch 09: Train Loss=1.0471, Val Loss=1.0377, Acc=0.5633, Precision=0.1853, Recall=0.8314, F1=0.

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

# ========== Feature Engineering ==========
angvel_diff = angvel_all - neighAngVel_all
pos_magnitude = np.sqrt(posx_all**2 + posy_all**2)
normalized_angvel = angvel_all / (radius_all + 1e-6)

# Stack features (original + engineered)
X = np.stack([
    angvel_all,
    neighAngVel_all,
    radius_all,
    posx_all,
    posy_all,
    angvel_diff,
    pos_magnitude,
    normalized_angvel
], axis=1)
y = rigid_all

# ========== Subset and Normalize ==========
subset_size = 1_000_000
indices = np.random.choice(len(X), size=subset_size, replace=False)
X = X[indices]
y = y[indices]

scaler = StandardScaler()
X = scaler.fit_transform(X)

# Stratified train-val split
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Convert to tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_val   = torch.tensor(X_val, dtype=torch.float32)
y_val   = torch.tensor(y_val, dtype=torch.float32).view(-1, 1)

# Class imbalance weight
pos_weight_value = ((len(y_train) - y_train.sum()) / y_train.sum()).item()
pos_weight = torch.tensor(pos_weight_value, dtype=torch.float32)

# ========== Define Neural Network ==========
class RigidClusterNet(nn.Module):
    def __init__(self):
        super(RigidClusterNet, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(8, 128),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(),
            nn.Dropout(0.3),

            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.LeakyReLU(),
            nn.Dropout(0.3),

            nn.Linear(64, 32),
            nn.LeakyReLU(),
            nn.Linear(32, 1)  # logits only
        )

    def forward(self, x):
        return self.net(x)

# Model setup
model = RigidClusterNet()
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)

# DataLoader for batch training
train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=2048, shuffle=True)

# ========== Training Loop ==========
epochs = 25
for epoch in range(epochs):
    model.train()
    total_loss = 0.0

    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        logits = model(batch_X)
        loss = criterion(logits, batch_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * batch_X.size(0)

    train_loss = total_loss / len(train_loader.dataset)

    # Validation
    model.eval()
    with torch.no_grad():
        val_logits = model(X_val)
        val_output = torch.sigmoid(val_logits)
        val_loss = criterion(val_logits, y_val)
        val_pred = (val_output > 0.5).float()
        val_acc = (val_pred == y_val).float().mean().item()

        tp = ((val_pred == 1) & (y_val == 1)).sum().item()
        fp = ((val_pred == 1) & (y_val == 0)).sum().item()
        fn = ((val_pred == 0) & (y_val == 1)).sum().item()

        precision = tp / (tp + fp + 1e-6)
        recall = tp / (tp + fn + 1e-6)
        f1 = 2 * precision * recall / (precision + recall + 1e-6)
        auc = roc_auc_score(y_val.numpy(), val_output.numpy())

    scheduler.step(val_loss)

    print(f"Epoch {epoch+1:02d}: Train Loss={train_loss:.4f}, Val Loss={val_loss.item():.4f}, "
          f"Acc={val_acc:.4f}, Precision={precision:.4f}, Recall={recall:.4f}, "
          f"F1={f1:.4f}, AUC={auc:.4f}")

Epoch 01: Train Loss=1.0703, Val Loss=1.0475, Acc=0.5606, Precision=0.1842, Recall=0.8315, F1=0.3016, AUC=0.7303
Epoch 02: Train Loss=1.0535, Val Loss=1.0446, Acc=0.5642, Precision=0.1848, Recall=0.8268, F1=0.3021, AUC=0.7316
Epoch 03: Train Loss=1.0499, Val Loss=1.0435, Acc=0.5661, Precision=0.1854, Recall=0.8257, F1=0.3028, AUC=0.7322
Epoch 04: Train Loss=1.0480, Val Loss=1.0419, Acc=0.5650, Precision=0.1854, Recall=0.8285, F1=0.3030, AUC=0.7324
Epoch 05: Train Loss=1.0479, Val Loss=1.0421, Acc=0.5517, Precision=0.1828, Recall=0.8436, F1=0.3004, AUC=0.7328
Epoch 06: Train Loss=1.0461, Val Loss=1.0419, Acc=0.5596, Precision=0.1843, Recall=0.8346, F1=0.3019, AUC=0.7327
Epoch 07: Train Loss=1.0454, Val Loss=1.0414, Acc=0.5596, Precision=0.1840, Recall=0.8324, F1=0.3014, AUC=0.7324
Epoch 08: Train Loss=1.0453, Val Loss=1.0407, Acc=0.5614, Precision=0.1844, Recall=0.8306, F1=0.3017, AUC=0.7331
Epoch 09: Train Loss=1.0445, Val Loss=1.0406, Acc=0.5579, Precision=0.1840, Recall=0.8372, F1=0.